# Freight Operations Platform - Getting Started

This notebook demonstrates the core functionality of the freight operations platform.

## Topics Covered

1. Load data model and calculations
2. Rate per mile analysis
3. Deadhead optimization
4. Profitability assessment
5. Basic agent integration

In [ ]:
# Import required libraries
import sys
from pathlib import Path
from datetime import datetime, timedelta
from decimal import Decimal

# Add project root to path
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

from src.data.models.load import Load, Location, LoadStatus, LoadType

print("✅ Imports successful")

## 1. Creating a Load

Let's create a sample load from Dallas, TX to Los Angeles, CA.

In [ ]:
# Define locations
origin = Location(
    city="Dallas",
    state="TX",
    zip_code="75001",
    latitude=32.7767,
    longitude=-96.7970
)

destination = Location(
    city="Los Angeles",
    state="CA",
    zip_code="90001",
    latitude=34.0522,
    longitude=-118.2437
)

# Create load
load = Load(
    load_id="DAT-123456",
    broker_name="Reliable Freight Brokers",
    broker_mc="MC654321",
    origin=origin,
    destination=destination,
    pickup_date=datetime.now() + timedelta(days=1),
    delivery_date=datetime.now() + timedelta(days=3),
    commodity="Auto Parts",
    weight=8000,
    rate=Decimal("3600.00"),
    fuel_surcharge=Decimal("180.00"),
    loaded_miles=1440,
    deadhead_miles=50,
    load_type=LoadType.FULL
)

print(f"Created load: {load.load_id}")
print(f"Route: {load.origin} → {load.destination}")
print(f"Distance: {load.loaded_miles} miles")

## 2. Rate Per Mile Analysis

Calculate the rate per mile and assess profitability.

In [ ]:
print("=" * 60)
print("RATE ANALYSIS")
print("=" * 60)
print(f"Base Rate:         ${load.rate:>10,.2f}")
print(f"Fuel Surcharge:    ${load.fuel_surcharge:>10,.2f}")
print(f"Additional:        ${load.additional_charges:>10,.2f}")
print("-" * 60)
print(f"Gross Revenue:     ${load.gross_revenue:>10,.2f}")
print()
print(f"Loaded Miles:      {load.loaded_miles:>10,} mi")
print(f"Deadhead Miles:    {load.deadhead_miles:>10,} mi")
print(f"Total Miles:       {load.total_miles:>10,} mi")
print()
print(f"Rate/Loaded Mile:  ${load.rate_per_mile:>10.2f}")
print(f"Rate/All Miles:    ${load.all_miles_rate:>10.2f}")
print(f"Deadhead %:        {load.deadhead_percentage:>10.1f}%")
print("=" * 60)

## 3. Profitability Assessment

Check if the load meets our acceptance criteria.

In [ ]:
# Define acceptance criteria
MIN_RPM = Decimal("2.00")  # Minimum $2.00 per mile
MAX_DEADHEAD = 15.0        # Maximum 15% deadhead

# Check profitability
is_profitable = load.is_profitable(MIN_RPM, MAX_DEADHEAD)

print("=" * 60)
print("PROFITABILITY CHECK")
print("=" * 60)
print(f"Minimum RPM Required:    ${MIN_RPM:.2f}")
print(f"Actual RPM:              ${load.rate_per_mile:.2f}")
print(f"Meets RPM?               {'✅ YES' if load.rate_per_mile >= MIN_RPM else '❌ NO'}")
print()
print(f"Max Deadhead Allowed:    {MAX_DEADHEAD:.1f}%")
print(f"Actual Deadhead:         {load.deadhead_percentage:.1f}%")
print(f"Meets Deadhead?          {'✅ YES' if load.deadhead_percentage <= MAX_DEADHEAD else '❌ NO'}")
print()
print(f"Overall Decision:        {'✅ ACCEPT' if is_profitable else '❌ REJECT'}")
print("=" * 60)

## 4. Cost Analysis

Calculate estimated costs and profit margin.

In [ ]:
# Operating costs (from config)
COST_PER_MILE = Decimal("1.50")  # Total operating cost per mile
DRIVER_PAY_PCT = Decimal("0.70")  # Driver gets 70%

# Calculate costs
total_cost = Decimal(load.total_miles) * COST_PER_MILE
driver_pay = load.gross_revenue * DRIVER_PAY_PCT
net_profit = load.gross_revenue - driver_pay - total_cost
profit_margin = (net_profit / load.gross_revenue) * 100 if load.gross_revenue > 0 else 0

print("=" * 60)
print("COST ANALYSIS")
print("=" * 60)
print(f"Gross Revenue:     ${load.gross_revenue:>10,.2f}")
print()
print("Costs:")
print(f"  Operating Cost:  ${total_cost:>10,.2f}  ({load.total_miles} mi × ${COST_PER_MILE})")
print(f"  Driver Pay:      ${driver_pay:>10,.2f}  ({DRIVER_PAY_PCT*100:.0f}% of revenue)")
print("-" * 60)
print(f"Net Profit:        ${net_profit:>10,.2f}")
print(f"Profit Margin:     {profit_margin:>10.1f}%")
print("=" * 60)

## 5. Multiple Load Comparison

Let's create several loads and compare them.

In [ ]:
# Create multiple loads for comparison
loads = [
    Load(
        load_id="LOAD-001",
        broker_name="Broker A",
        origin=Location(city="Dallas", state="TX"),
        destination=Location(city="Houston", state="TX"),
        pickup_date=datetime.now() + timedelta(days=1),
        delivery_date=datetime.now() + timedelta(days=1, hours=6),
        commodity="Equipment",
        weight=5000,
        rate=Decimal("650.00"),
        loaded_miles=240,
        deadhead_miles=15
    ),
    Load(
        load_id="LOAD-002",
        broker_name="Broker B",
        origin=Location(city="Dallas", state="TX"),
        destination=Location(city="Denver", state="CO"),
        pickup_date=datetime.now() + timedelta(days=1),
        delivery_date=datetime.now() + timedelta(days=2),
        commodity="Electronics",
        weight=3000,
        rate=Decimal("1800.00"),
        loaded_miles=780,
        deadhead_miles=120,
        is_expedited=True
    ),
    Load(
        load_id="LOAD-003",
        broker_name="Broker C",
        origin=Location(city="Dallas", state="TX"),
        destination=Location(city="Phoenix", state="AZ"),
        pickup_date=datetime.now() + timedelta(days=2),
        delivery_date=datetime.now() + timedelta(days=3),
        commodity="Construction Materials",
        weight=10000,
        rate=Decimal("2400.00"),
        fuel_surcharge=Decimal("120.00"),
        loaded_miles=1070,
        deadhead_miles=25
    ),
]

print("=" * 80)
print("LOAD COMPARISON")
print("=" * 80)
print(f"{'Load ID':<12} {'Route':<25} {'Miles':<8} {'DH%':<8} {'RPM':<10} {'Revenue':<12} {'Accept?'}")
print("-" * 80)

for ld in loads:
    route = f"{ld.origin.city}-{ld.destination.city}"
    accept = "✅" if ld.is_profitable(MIN_RPM, MAX_DEADHEAD) else "❌"
    
    print(
        f"{ld.load_id:<12} "
        f"{route:<25} "
        f"{ld.loaded_miles:<8} "
        f"{ld.deadhead_percentage:<8.1f} "
        f"${ld.rate_per_mile:<9.2f} "
        f"${ld.gross_revenue:<11,.2f} "
        f"{accept}"
    )

print("=" * 80)

## Next Steps

1. **Explore Agents**: Check out the agent implementations in `src/agents/`
2. **MCP Tools**: Review tool integrations in `src/tools/`
3. **Configuration**: Customize `config/config.yaml` for your business
4. **Data Analysis**: Create notebooks for historical rate analysis
5. **Testing**: Run the test suite with `pytest tests/`

## Resources

- [Project Documentation](../README.md)
- [Claude Code Instructions](../CLAUDE.md)
- [Configuration Guide](../config/config.yaml)